# Pumped Catchment eDNA metabarcoding data processing

I will use [metaBEAT](https://github.com/HullUni-bioinformatics/metaBEAT), a tool tailored towards reproducible and efficient analyses of metabarcoding data that was developed by Dr. Christoph Hahn (University of Graz) for the EvoHull group at University of Hull. The pipeline is still under active development and will likely be extended further in the future. The pipeline is available in a Docker container with all necessary dependencies. The Docker image builds on [ReproPhylo](https://github.com/HullUni-bioinformatics/ReproPhylo).

The metaBEAT tool is designed for complete bioinformatic analysis from raw data, and performs (optionally) de-multiplexing, quality filtering, chimera detection, clustering, and taxononomic assignment (outputs in `.biom` and `.tsv` formats). It currently supports BLAST, Kraken and phylogenetic placement (pplacer).

# Data input

This notebook will perform basic processing (trimming, merging, chimera removal, clustering and taxonomic assignment) of the metabarcoding data.

Minimum input for an analysis is a set of query sequences in one or several files (a number of file formats are accepted, e.g. `.fasta`, `.fastq`). These will be run through the pipeline sequentially.

Information on the nature and location of the query sequence files must be provided in a separate tab-delimited text file via the -Q flags.

Each line in this text file should look as follows: unique sample_ID, format, file1, file2

The required text files can be generated in any text editor. So theoretically, nano could be used in the terminal to construct the text file. For reproducibility and ease, a simple program can be used to generate the required file.

In the cell below, it is produced using a simple python script. The script will list all files in the location to which you downloaded your Illumina data (specified via the 'datadir' variable). It assumes that there is a file ending in `_R1.fastq` for each sample. For each such file, it will extract the sample name from the filename and format the required line for the text file accordingly. The resulting file is called `Querymap.txt` (specified in the 'to' variable).

In [ ]:
!pwd

In [ ]:
!mkdir 1-trimming

In [ ]:
cd 1-trimming

In [ ]:
!ls -1 ../raw_reads/

Prepare a text file specifying the samples to be processed, including the format and location of the reads - the querymap.

The next command expects two `.fastq` files (forward and reverse reads) per sample in the directory `../raw_reads/`. It expects the files to be named 'SampleID', followed by '_R1' or '_R2' to identify the forward/reverse read file respectively.

The raw data have been downloaded and demultiplexed. They can be found in `../raw_reads/`.

SampleID must correspond to the first column in a file called `Sample_accessions.tsv`. This will either be pre-made to correspond to downloading read data from the NCBI Sequence Read Archive, or you will have to make it. The marker is '12S'.

If the `Sample_accessions.tsv` was pre-made, use this code to proceed:

In [ ]:
#%%bash
#
#for a in $(cat ../Data/Sample_accessions.tsv | grep "12S" | cut -f 1 | grep "SampleID" -v)
#do
#    R1=$(ls -1 ../raw_reads/$a-12S_* | grep "_R1.fastq")
#    R2=$(ls -1 ../raw_reads/$a-12S_* | grep "_R2.fastq")
#
#    echo -e "$a\tfastq\t$R1\t$R2"
#done > Querymap.txt

In [ ]:
!head -n 10 Querymap.txt

**OR...**

To make the `Sample_accessions.tsv` file, use:

In [ ]:
!echo "SampleID" > ../1-trimming/Sample_accessions.tsv

In [ ]:
%%bash
for a in $(ls ../raw_reads/ | grep -w "R1" | cut -d '.' -f 1)
do 
   SampleID=$a
   
   echo -e "$SampleID"
done >> ../1-trimming/Sample_accessions.tsv

In [ ]:
!cat ../1-trimming/Sample_accessions.tsv

In [ ]:
%%bash

for a in $(cat ../1-trimming/Sample_accessions.tsv | grep "SampleID" -v)
do
    R1=$(ls -1 ../raw_reads/$a.* | grep -w "R1")
    R2=$(ls -1  ../raw_reads/$a.* | grep -w "R2")

    echo -e "$a\tfastq\t$R1\t$R2"
done > Querymap.txt

In [ ]:
!head -n 10 Querymap.txt

To the `Querymap.txt` file, add two columns which specify the number of bases to remove from the forward and reverse read. In our case, we want to remove 18 bp to ensure that there is no forward or reverse primer left.

In [ ]:
%%bash

sed 's/$/&\t18/' Querymap.txt > Querymap_new.txt

Have a look (note that the output is probably line-wrapped):

In [ ]:
!head -n 4 Querymap_new.txt

In [ ]:
%%bash

sed 's/$/&\t18/' Querymap_new.txt > Querymap_final.txt

In [ ]:
!head -n 4 Querymap_final.txt

# Raw read processing

Now, perform basic quality trimming and clipping (Trimmomatic) and paired-end read merging (flash). metaBEAT will be used to process all samples in one go.

In [ ]:
!metaBEAT_global.py -h

Command to trim:

In [ ]:
%%bash

echo -e "Starttime: $(date)\n"

metaBEAT_global.py \
-Q Querymap_final.txt \
--trim_qual 30 \
--read_crop 110 \
--trim_minlength 90 \
--merge \
--product_length 106 \
--forward_only \
--length_filter 106 \
--length_deviation 0.2 \
-m 12S -o Eel2017_trim30-min90-crop110-forwonly-filter100-deviation0.2 \
-n 5 -v \
-@ youremail@server.com &> log

echo -e "Endtime: $(date)\n"

Read processing will take several hours.


# Visualise query survival after trimming

metaBEAT will generate a directory with all temporary files that were created during the processing for each sample and will record useful stats summarizing the data processing in the file `metaBEAT_read_stats.csv`. You can explore the table manually or quickly plot out some of these stats here:

In [ ]:
%matplotlib inline
import pandas as pd

df = pd.read_csv('Eel2017_trim30-min90-crop110-forwonly-filter100-deviation0.2_read_stats.csv', index_col=0)
df['fraction'] = df['queries']/(df['total']*0.5)
df.fraction.hist(bins=50)

Detailed information on what metaBEAT did to each sample is contained in the `log` file. It contains the exact commands that were run for each sample during each step of the process.

In [ ]:
!head -n 100 log

The next steps in the processing will be chimera detection, and global clustering of the centroids from all clusters from all samples to produce denovo OTUs. The temporary files from the global clustering and the final OTU table were written to the directory `./GLOBAL`.

In [ ]:
!ls GLOBAL/

The denovo OTU table (numbers are reads) can be viewed to see how OTUs are distributed across your samples. 

# Chimera detection

Some stats on the read counts before/after trimming, merging etc. are summarised for you in `metaBEAT_read_stats.csv`.

Next stage of the processing is chimera detection and removal of putative chimeric sequences. We'll do that using uchime as implemented in vsearch.

In [ ]:
!pwd

In [ ]:
cd ..

In [ ]:
!mkdir 2-chimera_detection

In [ ]:
cd 2-chimera_detection

Convert reference database from GenBank to fasta format to be used in chimera detection.

Prepare `REFmap.txt` file, i.e. text file that specifies the location and the format of the reference to be used.
The reference sequences in GenBank format are present in subdirectories for each vertebrate group in the `../Reference_database` directory.

In [ ]:
!echo '../supplementary_data/Reference_DBs/12S_Fish_SATIVA_cleaned_May_2017.gb\tgb\n' \
'../supplementary_data/Reference_DBs/M.zebra.gb\tgb'> REFmap.txt

In [ ]:
!cat REFmap.txt

In [ ]:
!metaBEAT_global.py -h

In [ ]:
%%bash

metaBEAT_global.py \
-R REFmap.txt \
-f \
-@ youremail@server.com

This will produce `refs.fasta`.

In [ ]:
!head refs.fasta

Now run chimera detection.

In [ ]:
%%bash


for a in $(cut -f 1 ../1-trimming/Querymap.txt)
do
    if [ -s ../1-trimming/$a/$a\_trimmed.fasta ]
    then
        echo -e "\n### Detecting chimeras in $a ###\n"
        mkdir $a
        cd $a
        vsearch --uchime_ref ../../1-trimming/$a/$a\_trimmed.fasta --db ../refs.fasta \
        --nonchimeras $a-nonchimeras.fasta --chimeras $a-chimeras.fasta &> log 
        cd ..

    else
        echo -e "$a is empty"
    fi
done

# Clustering and taxonomic assignment against UK fish database

In [ ]:
!pwd

In [ ]:
cd ..

In [ ]:
!mkdir 3-taxonomic_assignment_fish

In [ ]:
cd 3-taxonomic_assignment_fish/

Produce the text file containing the fish reference sequences using the command line - we call it `REFmap.txt`.

In [ ]:
!echo '../supplementary_data/Reference_DBs/12S_Fish_SATIVA_cleaned_May_2017.gb\tgb\n' \
'../supplementary_data/Reference_DBs/M.zebra.gb\tgb'> REFmap.txt

In [ ]:
!cat REFmap.txt

Produce the text file containing non-chimera query sequences - `Querymap.txt`.

In [ ]:
%%bash

#Querymap
for a in $(ls -l ../2-chimera_detection/ | grep "^d" | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
    echo -e "$a-nc\tfasta\t../2-chimera_detection/$a/$a-nonchimeras.fasta"
done > Querymap.txt

In [ ]:
!cat Querymap.txt

The Querymap.txt file has been made but includes the `./GLOBAL` directory in which all centroids and queries are contained. This will cause metaBEAT to fail so it must be removed manually from the `Querymap.txt` file.

In [ ]:
!sed '/GLOBAL/d' Querymap.txt > Querymap_final.txt

In [ ]:
!cat Querymap_final.txt

Update the taxonomy database in the current metaBEAT image.

In [ ]:
!taxit new_database \
--taxdump-url ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump_archive/taxdmp_2018-10-01.zip \
    --download-dir /usr/bin/ /usr/bin/taxonomy.db

That's almost it. Now start the pipeline to do sequence clustering and taxonomic assignment of non-chimera queries via metaBEAT. As input, `Querymap.txt` containing samples that have been trimmed, merged and checked for chimeras, and the `REFmap.txt` file must be specified. metaBEAT will be asked to attempt taxonomic assignment using BLAST.

metaBEAT will automatically wrangle the data into the particular file formats that are required by each of the methods, run all necessary steps, and finally convert the outputs of each program to a standardized BIOM table.

GO!

In [ ]:
!metaBEAT_global.py -h

In [ ]:
%%bash

echo -e "Starttime: $(date)\n"

metaBEAT_global.py \
-Q Querymap_final.txt \
-R REFmap.txt \
--cluster --clust_match 1 --clust_cov 3 \
--blast --min_ident 0.98 \
-m 12S -n 5 \
-E -v \
-@ youremail@server.com \
-o Eel2017_12S-trim30-min90-crop110-mergeforwonly-filt100-dev0.2_nonchimera_c1cov3_blast0.98_fish &> log_fish

echo -e "Endtime: $(date)\n"

In [ ]:
!tail -n 50 log_fish

**DONE, Output file under GLOBAL/BLAST_0.98